In [1]:
# sample program for viterbi decoding
# example borrowed from https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/slides/sequence-slides.pdf
import numpy as np

In [14]:
def viterbi_decoding(emission_probs, transition_probs, bos_out):
    """Viterbi algorithm that returns best scored sequence output.
    
    Args:
        emission_probs: [tag_num x l]
        transition_probs: [tag_num x tag_num]
        bos_out: [tag_num]
        
        *NOTE: above probs are all in log space, so chain rule
            works as addition instead of multiplication
    Returns:
        best_score: float
        best_sequence: [tag_num]
    """
    tag_num, l = emission_probs.shape
    v = np.zeros([tag_num, l])
    
    
    # initialize the first step to be emission + transition from <bos>
    v[:, 0] = bos_out + emission_probs[:, 0]
    
    # keep track of sequence
    sequence_bptr = []
    
    for t in range(1, l):
        tmp_sum = v[:, t-1] + transition_probs
        best_scores = tmp_sum.max(-1)
        best_pred = tmp_sum.argmax(-1)
        
        v[:, t] = best_scores + emission_probs[:, t]
        sequence_bptr.append(best_pred)
        
        
    best_score = max(v[:, l - 1])
    best_seq = [None for _ in range(l)]
    cur_ptr = v[:, l - 1].argmax()
    
#     best_seq[-1] = cur_ptr
    for t in range(l - 1, 0, -1):
        predecessor = sequence_bptr[t - 1][cur_ptr]
        best_seq[t] = cur_ptr
        cur_ptr = predecessor
        
    best_seq[0] = predecessor
    return best_score, best_seq

In [15]:
emission_probs = np.array([
    [-2, -10], [-3, -1], [-3, -3]
]).transpose()

transition_probs = np.array([
        [-3, -1],
        [-1, -3]
    ])
bos_out = np.array([-1, -2])

In [16]:
b_score, b_seq = viterbi_decoding(emission_probs, transition_probs, bos_out)

In [17]:
b_seq

[0, 1, 0]

In [18]:
b_score

-9.0